In [2]:
import glob
import os
import time
import json 
from datetime import date, datetime, timedelta
from dotenv import load_dotenv

from src.utils import get_config, get_rfc_date, get_key_config, is_working, dump_file
from src.thirdparty.youtube_fn import YouTubeAPI
from src.wrapper import youtube_summary_wrapper


In [3]:
## time elapse check
load_dotenv()

target_start_date = "2018-01-01"
args = {'option':'youtube', 
        'youtube_option':'query', 
        'get_results':10, 
        'date_before':7, 
        'target_date':target_start_date, 
        'aws_env':False, 
        'num_of_comments':3, 
        'day_of_the_week':'Mon'}

config = {
            "LLM" : {
                "max_tokens" : 2000,
                "temperature" : 0.0
            },
            "Channel" : {
                "channel_id" : [
                    "UCEjOSbbaOfgnfRODEEMYlCw",
                    "UCj9yUGuMVVdm2DqyvJPUeUQ",
                    "UCvJJ_dzjViJCoLf5uKUTwoA",
                    "UCjzi56cxvmEDwjo1Bd2Yxpg",
                    "UCUBHuG2jnmRYu8AyKuGGing",
                    "UC32lvHvH7uAeKM7ybVwl2JQ",
                    "UCDLCUoNT_y-CCgiBAHNNuRg",
                    "UCvADSaOPH2z9NNQY-6Sn_FA",
                    "UC675NhQ4EU5TzwCMwYp5XCw",
                    "UCrp_UI8XtuYfpiqluWLD7Lw"
                ],
                "keywords":[
                    "LITHIUM PRICE",
                    "LITHIUM GLOBAL DEMAND",
                    "LITHIUM SUPPLY CHAIN",
                    "LITHIUM INVESTMENT"

                ]
            },
            "Query" : {
                "query" : ['"SK On"', 'SK On', 'SK Battery', 'Battery SK']
            }
        }

In [4]:
target_start_date = '2018-01-01'
[publishedAfter, publishedBefore] = get_rfc_date(date=datetime.strptime(target_start_date, "%Y-%m-%d"), before_days=7)

key_config = {}
key_config['Anthropic'] = {'key':os.environ.get('ANTHROPIC_API')}
key_config['youtube_api'] = {'key':os.environ.get('YOUTUBE_API')}
# print(key_config)


youtube = YouTubeAPI(
    api_key = key_config['youtube_api']['key'],
    service_name="youtube",
    version="v3",
    publishedAfter=publishedAfter,
    publishedBefore=publishedBefore        
)

In [5]:
# wrapper class
def get_infos(target_date, args, config):
    # print('args exist in get_infos:', args)
    if target_date == 'now':
        _videos = str(date.today().strftime('%Y-%m-%d') + '.json')
    else:
        _videos = str(target_date.strftime('%Y-%m-%d') + '.json')

    ## search videos from youtube
    if os.path.exists(_videos):
        with open(_videos, 'r', encoding='utf-8') as json_file:
            video_dict = json.load(json_file)
    else:
        video_dict = youtube.get_video_infos(args, config)

        for category in video_dict.keys():
            for video_info in video_dict[category]:
                video_info['details'], video_info['duration'] = youtube.get_video_duration(video_info['id']['videoId'])
                video_info['caption'] = youtube.get_caption(video_info['id']['videoId'])
                video_info['statistics'] = youtube.get_statistics(video_info['id']['videoId'])
                video_info['comments'] = youtube.get_comments(video_info['id']['videoId'])

        dump_file(video_dict, target_date=target_date, aws_env=args['aws_env'])
        
    return print(f'file {target_date.strftime("%Y-%m-%d")}.json saved!')

In [17]:
start_date = "2024-05-27"
num_of_files = 1

for i in range(num_of_files):
    if i == 0:
        target_start_date = start_date
    else:
        target_start_date = start_date
        date_obj = datetime.strptime(target_start_date, "%Y-%m-%d")
        delta = timedelta(days=7)
        target_start_date = date_obj + delta*(i)
        target_start_date = target_start_date.strftime('%Y-%m-%d')

        [publishedAfter, publishedBefore] = get_rfc_date(date=datetime.strptime(target_start_date, "%Y-%m-%d"), before_days=7)

        youtube = YouTubeAPI(
            api_key = key_config['youtube_api']['key'],
            service_name="youtube",
            version="v3",
            publishedAfter=publishedAfter,
            publishedBefore=publishedBefore        
        )

    get_infos(datetime.strptime(target_start_date, "%Y-%m-%d"), args, config)

file 2024-05-27.json saved!
